In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import transformers
import datasets
from datasets import load_dataset
import tensorflow_datasets as tensorflow_datasets
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, TFT5ForConditionalGeneration
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
data_dir='./data'
log_dir='./logs'
save_path='./models'
cache_train='/cache/t5.train'
cache_test='/cache/t5.test'

In [11]:
tokenizer=AutoTokenizer.from_pretrained("cache/huggingface/transformers")

file cache/huggingface/transformers\config.json not found


OSError: Can't load config for 'cache/huggingface/transformers'. Make sure that:

- 'cache/huggingface/transformers' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'cache/huggingface/transformers' is the correct path to a directory containing a config.json file

